In [1]:
# Miners
from pm4py import serialize, deserialize
from pm4py import discover_dfg as dfg_discovery

from pm4py.algo.discovery.alpha import algorithm as alpha_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.discovery.inductive import algorithm as inductive_miner


# Evaluators
import pandas as pd
import re
import pm4py
import sys
import dask
from pm4py.discovery import DFG
from pm4py import discover_dfg_typed

from contribution import fitness_alignment, generalization, precision_alignment

In [2]:
data = pd.read_csv('BPI_2012.csv')
# log = pm4py.read_xes('BPI Challenge 2017.xes')

In [3]:
# dataframe = dataframe.rename(columns={"Incident ID": "case:concept:name", "IncidentActivity_Type": "concept:name", "DateStamp": "time:timestamp"})

In [4]:
groups = data['case:concept:name'].unique()

In [31]:
dataframe = data[data['case:concept:name'].isin(groups[:80])][:]

In [32]:
dataframe['time:timestamp'] = pd.to_datetime(dataframe['time:timestamp'], utc=True)
dataframe['concept:name'] = dataframe['concept:name'].astype(str)
dataframe['case:concept:name'] = dataframe['case:concept:name'].astype(str)

# data['time:timestamp'] = pd.to_datetime(data['time:timestamp'], utc=True)
# data['concept:name'] = data['concept:name'].astype(str)
# data['case:concept:name'] = data['case:concept:name'].astype(str)

In [33]:
dfg, start_activities, end_activities = dfg_discovery(dataframe)

In [34]:
theDfg = DFG(dfg, start_activities=start_activities, end_activities=end_activities)

In [35]:
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
from pm4py.algo.evaluation.generalization import algorithm as generalization_pm4py

In [36]:
net, im, fm = heuristics_miner.apply_dfg(dfg)

In [39]:
dgen = dask.delayed(generalization_pm4py.apply)(dataframe, net, im, fm)

In [37]:
generalization_output = generalization_pm4py.apply(dataframe, net, im, fm)

replaying log with TBR, completed variants ::   0%|          | 0/52 [00:00<?, ?it/s]

In [38]:
generalization_output

0.7497700473915065

In [43]:
dgen.compute()

replaying log with TBR, completed variants ::   0%|          | 0/52 [00:00<?, ?it/s]

0.7497700473915065

In [25]:
gen_80 = generalization.apply(dataframe, net, im, fm)

replaying log with TBR, completed variants ::   0%|          | 0/22 [00:00<?, ?it/s]

In [28]:
generalization.compute(gen_40['trans_occ_map'] + gen_80['trans_occ_map'], net)

0.7497700473915065

In [19]:
generalization.compute(gen_80['trans_occ_map'], net)

0.7497700473915065